In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_dir = '/mnt/c/Users/Sergi/Desktop/BSC/modelos_entrenados/transformers_rel2'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

path = '/mnt/c/Users/Sergi/Desktop/BSC/spanish_sapbert_models/sapbert_15_noparents_1epoch'
tokenizer = AutoTokenizer.from_pretrained(path)

/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
def compute_relation(source, target):
    """
    Computes relations between source and target entities using Transformers model.

    Parameters:
    source (list): List of source entities.
    target (list): List of target entities.

    Returns:
    list: List of labels representing computed relations.
    """
    final_labels = list()
    tokenized_mention = tokenizer(source, target, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokenized_mention)
    logits = output.logits
    for i in range(len(logits.tolist())):
        predscores = {label: score for label, score in zip(all_labels, logits.tolist()[i])}
        top_n_labels = sorted(predscores, key=predscores.get, reverse=True)[:1]
        filtered_labels = [label for label in top_n_labels if predscores[label] > -1]
        final_labels.append(filtered_labels)
    return final_labels

In [23]:
# import pandas as pd

# # Read the TXT file into a list of lines
# with open('/mnt/c/Users/Sergi/Desktop/BSC/Test_files/testset_rel_model2.txt') as f:
#     lines = f.readlines()

# # Split each line into a list of values
# data = []
# for line in lines:
#     values = line.strip().split('\t')
#     data.append(values)

# # Create a DataFrame from the list of data
# df = pd.DataFrame(data[1:], columns=data[0])

# # Save the DataFrame to a TSV file
# df.to_csv('/mnt/c/Users/Sergi/Desktop/BSC/Test_files/testset_rel_model.tsv', index=False, sep='\t')

In [4]:
import pandas as pd

path = '/mnt/c/Users/Sergi/Desktop/BSC/Test_files/testset_rel_model.tsv'
data = pd.read_csv(path, delimiter='\t')
source = data['source'].tolist()
target = data['target'].tolist()

In [ ]:
rels = compute_relation(source, target)

In [ ]:
rels